<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/10_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習で文書分類を試みる

* WikipediaからUSの男性俳優と女性俳優のページを2020年にクローリングして、spaCyで簡単な前処理を済ませてあるデータを使う。

 * 固有名詞、冠詞、前置詞、代名詞、副詞、数詞、接続詞は除去してある。
 * lemmatizationした結果を使っている。
 * "actor"と"actress"という単語は特別に除去してある。

* 分析の目的1: 検証データでできるかぎりチューニングを行い、最後にテストデータでの分類性能を明らかにする。

* 分析の目的2: 男性俳優と女性俳優のページを分類する際に、どのような単語が特に効いているかを調べる。

 * この調査によって、俳優に関する記述におけるジェンダー・ステレオタイプを明らかにできるか？

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer

%config InlineBackend.figure_format = 'retina'

## 1) データファイルを読み込む

* データファイルは、あらかじめ自分のGoogle Driveの適当なフォルダに置いておく。

* データファイルの各行には、女性俳優(1)か男性俳優(0)かを表すフラグ、俳優の名前、Wikipediaのページの本文が、この順に格納されている。

* データファイルの各行に対してeval組み込み関数を適用すると、Pythonのリストに変換できるようなフォーマットで、ファイルに記録されている。

In [ ]:
y = list()
names = list()
corpus = list()
with open('/content/drive/MyDrive/data/us_actors_and_actresses.txt', 'r') as f:
  for line in f:
    flag, name, text = eval(line.strip())
    y.append(int(flag))
    names.append(name)
    corpus.append(text)

* `corpus`は、本文の文字列がたくさん入っているリスト。最初の10件の文書だけ表示してみる。
 * ちなみにbearはbornの原型なので、多く含まれる。


In [ ]:
corpus[:10]

* `y`は、各文書が男性俳優に関するものか、女性俳優に関するものかを表す、0/1の値（これが目的変数）。



In [ ]:
y = np.array(y)
print(y.sum(), len(y))

* `names`は、各文書が誰に関する記事かを表す俳優の名前（分類には使わない参考データ）。

In [ ]:
names = np.array(names)
print(names[:20])

In [ ]:
N = len(y) # 全文書数
print(f'We have {N:d} documents.')

## 2) テストデータを分離

In [ ]:
names_train, names_test, y_train, y_test = train_test_split(list(zip(names, corpus)), y, test_size=0.2, random_state=42)

In [ ]:
names_train, corpus_train = zip(*names_train)
names_test, corpus_test = zip(*names_test)

In [ ]:
print(len(names_train), len(names_test))

**training setとtest setへの分割は、変えないようにしてください。**

## 3) TF-IDFで各文書をベクトル化する



### TF-IDFとは
* TF-IDFは単語列をベクトル化する方法のひとつ。
* ベクトルの次元は語彙数となる。各文書が1つのベクトルへ変換される。

 * すべての単語について、文書ごとに重みを計算する方法。
 * TFとは、ある文書のなかにその単語が何回出現するか、その回数。
 * DFとは、単語がいくつの文書に出現するか、その文書数。IDFはDFの逆数。
 * TF-IDFは、ざっくり言うと、TFとIDFの積。
 * 特定の文書に注目すると、その文書に出現する回数が多いほど、TF-IDFの値は大きくなる。しかし、たくさんの文書に出現する単語は、IDFが小さくなるので、その分、TF-IDFの値は小さくなる。

### sklearnの`TfidfVectorizer`について
* TfidfVectorizerのパラメータをチューニングしても構わないです。

* ここでTF-IDFの計算をするとき、テストデータは使っていないので、ズルはしていない。
* stop_words='english'とは、英語のストップワードは語彙から取り除く、という意味。
  * ストップワードとはthe, a, is, ofなど、ありふれすぎていて、文書分類などのタスクにはあまり効かない単語のこと。こういう単語は、特徴量削減の意味も含めて、しばしばあらかじめ削除しておく。

* min_dfは、その数より少ない文書にしか出現しない単語を削除する、という意味のパラメータ。希少な単語を削除するために使う。

* max_dfは、0から1の間の実数で指定すると、その割合より多い文書に出現する単語を削除する、という意味のパラメータ。ありふれた単語を削除するために使う。

In [ ]:
# TF-IDFにより、単なる単語列であった文書を、ベクトル化する。
# （ベクトル化してしまえば、テキストデータのような非構造化データも、
#  様々な機械学習手法の入力として使えるようになる。）

vectorizer = TfidfVectorizer(stop_words='english', min_df=50, smooth_idf=False, sublinear_tf=True)
X_train = vectorizer.fit_transform(corpus_train)
print('X_train: 文書数　{}, 語彙数 {}'.format(*X_train.shape))

* 語彙を取得する。
 * NumPyの配列に変換しておく。



In [ ]:
vocab = np.array(vectorizer.get_feature_names_out())

* 語彙の一部を見てみる（アルファベット順に並んでいるようだ）。



In [ ]:
print(vocab[1000:1010])

* 最初の文書をTF-IDFでベクトル化したらどうなったかを、見てみる。
 * スパースな表現になっている。



In [ ]:
print(type(X_train))
print(X_train[0])

## 4) ロジスティック回帰による２値分類と評価

* 交差検証のための準備

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

### 正則化のハイパーパラメータ

In [ ]:
for C in 10. ** np.arange(-2, 3):
  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):
    clf = LogisticRegression(C=C, solver='liblinear', random_state=123)
    clf.fit(X_train[train_index], y_train[train_index])
    score = clf.score(X_train[valid_index], y_train[valid_index])
    print(f'\tscore {score:.4f}')
    scores.append(score)
  print(f'mean validation accuracy: {np.array(scores).mean():.4f} for C={C:.2e}')

* L1正則化も試す。

In [ ]:
for C in 10. ** np.arange(-2, 3):
  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):
    clf = LogisticRegression(penalty='l1', C=C, solver='liblinear', random_state=123)
    clf.fit(X_train[train_index], y_train[train_index])
    score = clf.score(X_train[valid_index], y_train[valid_index])
    print(f'\tscore {score:.4f}')
    scores.append(score)
  print(f'mean validation accuracy: {np.array(scores).mean():.4f} for C={C:.2e}')

### TF-IDFの設定
* まずmin_dfを変えてみる。

In [ ]:
for min_df in [10, 20, 50, 100]:

  vectorizer = TfidfVectorizer(stop_words='english', min_df=min_df, smooth_idf=False, sublinear_tf=True)
  X_train = vectorizer.fit_transform(corpus_train)

  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):
    clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
    clf.fit(X_train[train_index], y_train[train_index])
    scores.append(clf.score(X_train[valid_index], y_train[valid_index]))
  print(f'mean validation accuracy: {np.array(scores).mean():.4f} for min_df={min_df} where voc size={X_train.shape[1]}')

In [ ]:
for min_df in [2, 4, 6, 8]:

  vectorizer = TfidfVectorizer(stop_words='english', min_df=min_df, smooth_idf=False, sublinear_tf=True)
  X_train = vectorizer.fit_transform(corpus_train)

  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):
    clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
    clf.fit(X_train[train_index], y_train[train_index])
    scores.append(clf.score(X_train[valid_index], y_train[valid_index]))
  print(f'mean validation accuracy: {np.array(scores).mean():.4f} for min_df={min_df} where voc size={X_train.shape[1]}')

* 次にmax_dfも変えてみる。

In [ ]:
for max_df in [1.0, 0.5, 0.4, 0.3, 0.2, 0.1]:

  vectorizer = TfidfVectorizer(stop_words='english', min_df=6, max_df=max_df, smooth_idf=False, sublinear_tf=True)
  X_train = vectorizer.fit_transform(corpus_train)

  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):
    clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
    clf.fit(X_train[train_index], y_train[train_index])
    scores.append(clf.score(X_train[valid_index], y_train[valid_index]))
  print(f'mean validation accuracy: {np.array(scores).mean():.4f} for max_df={max_df} where voc size={X_train.shape[1]}')

## 5) チューニングされたハイパーパラメータでロジスティック回帰の最終的な評価をおこなう

* LogisticRegression() のカッコ内には、自分で見つけた最善のセッティングを書き込む。
* 学習は、訓練データ全体（テストデータ以外の全体）を使っている。

In [ ]:
# 最も良かった設定を使って、訓練データ全体で再学習

vectorizer = TfidfVectorizer(stop_words='english', min_df=6, smooth_idf=False, sublinear_tf=True)
X_train = vectorizer.fit_transform(corpus_train)
print('# X_train: 文書数　{}, 語彙数 {}'.format(*X_train.shape))

# tf-idfの設定を変えたので、語彙も取得し直しておく。
vocab = np.array(vectorizer.get_feature_names_out())

clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)

# そして、最終的にテストデータで評価
X_test = vectorizer.transform(corpus_test)
print(f'test accuracy: {clf.score(X_test, y_test):.4f}')

### 参考: その他の評価尺度

In [ ]:
y_test_pred = clf.predict(X_test)
y_test_pred_proba = clf.predict_proba(X_test)

# Accuracy
from sklearn.metrics import accuracy_score
print('accuracy: {:.4f}'.format(accuracy_score(y_test, y_test_pred)))

# Area Under the Receiver Operating Characteristic Curve
from sklearn.metrics import roc_auc_score
print('ROC AUC: {:.4f}'.format(roc_auc_score(y_test, y_test_pred_proba[:,1])))

# 様々な評価尺度をまとめてレポート
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

# confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_test_pred, labels=clf.classes_)
print(cm)

# confusion matrixの可視化版
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot();

## 6) SVMによる２値分類と評価

* SVMについては、私からは実行例を示しません。各自、試行錯誤してください。

In [ ]:
# これは単純な実行例にすぎません。ハイパーパラメータのチューニングもしてください。
scores = list()
for train_index, valid_index in skf.split(X_train, y_train):
  svm = LinearSVC(random_state=123)
  svm.fit(X_train[train_index], y_train[train_index])
  scores.append(svm.score(X_train[valid_index], y_train[valid_index]))

print(f'mean validation accuracy: {np.array(scores).mean():.4f}')

## 7) チューニングされたハイパーパラメータを使って、SVMの最終的な評価をおこなう

* 試行錯誤をおこなってから、次のセルを実行してください。

In [ ]:
# 最も良かった設定を使って、訓練データ全体で再学習

svm = LinearSVC(random_state=123)
svm.fit(X_train, y_train)

X_test = vectorizer.transform(corpus_test)
print(f'test accuracy: {svm.score(X_test, y_test):.4f}')

## 8) ロジスティック回帰による分類に効いている単語を調べる

* 訓練データが最も数が多いので、訓練データの分類に最も効いている単語100語を調べる。

* 下に示すのは、あくまで一つの方法にすぎない。他にどんな方法があるか調べて使ってみよう。

 * 下の手法の欠点は、男性俳優の文書に特徴的な単語と、女性俳優の文書に特徴的な単語とを、区別できない点である。

 * ヒント： 「svm important features」 あたりでググってみる。

### recursive feature eliminationという特徴量選択の手法
* https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html



In [ ]:
from sklearn.feature_selection import RFE

# 最も良かった設定を使って、分類器のインスタンスを準備
clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)

# RFEで重要な属性上位100個を抽出
rfe = RFE(estimator=clf, n_features_to_select=100, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

### ロジスティック回帰の係数をそのまま可視化する

* 正の係数と負の係数で、それぞれ絶対値が大きいもの30個を可視化する。

In [ ]:
clf = LogisticRegression(C=1.0, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)

n_features = 30
positions = np.arange(n_features)[::-1]

* femaleクラスのほうに効いている単語
 * 必ずしも女性だけに関係するわけではない単語が含まれているか？


In [ ]:
indices = np.argsort(- clf.coef_[0])[:n_features]
widths = clf.coef_[0][indices]
yticks = vocab[indices]

plt.figure(figsize=(12,9))
plt.barh(positions, widths, align='center') 
plt.yticks(positions, yticks) 
plt.xlabel('coefficients')
plt.title(f'Coefficients of {n_features} Features Using Logistic Regression');

* maleクラスのほうに効いている単語
 * 必ずしも男性だけに関係するわけではない単語が含まれているか？


In [ ]:
indices = np.argsort(clf.coef_[0])[:n_features]
widths = clf.coef_[0][indices]
yticks = vocab[indices]

plt.figure(figsize=(12,9))
plt.barh(positions, widths, align='center') 
plt.yticks(positions, yticks) 
plt.xlabel('coefficients')
plt.title(f'Coefficients of {n_features} Features Using Logistic Regression');

## 9) SVMによる分類に効いている単語を調べる

* SVMについては実行例を示しません。上を参考に各自試してみてください。